In [15]:
import pandas as pd
import pandasql as ps
import plotly.express as px

In [16]:
crime_data=pd.read_csv('Crime_Data_from_2020_to_Present.csv')
crime_data.rename(columns={'Date Rptd':'Date'},inplace=True)
crime_data[['Month','Day','Year']]=crime_data.Date.str.split('/',expand=True)

crime_data[['Year_EXP','Time','AM_PM']]=crime_data.Year.str.split(' ',expand=True)
crime_data.drop(columns=['Date','Year'],inplace=True)
crime_data.rename(columns={'Year_EXP':'Year'},inplace=True)
crime_data

,DR_NO,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,...,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Month,Day,Year,Time,AM_PM
0,10304468,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,...,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978,01,08,2020,12:00:00,AM
1,190101086,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,...,NaN,700 S HILL ST,NaN,34.0459,-118.2545,01,02,2020,12:00:00,AM
2,201220752,09/16/2020 12:00:00 AM,1230,12,77th Street,1259,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),2004 1820 0913 0329 1202,...,NaN,700 E 73RD ST,NaN,33.9739,-118.2630,09,16,2020,12:00:00,AM
3,191501505,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,...,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019,01,01,2020,12:00:00,AM
4,191921269,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,...,NaN,14400 TITUS ST,NaN,34.2198,-118.4468,01,01,2020,12:00:00,AM
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276579,210404776,01/19/2021 12:00:00 AM,1600,4,Hollenbeck,497,1,510,VEHICLE - STOLEN,NaN,...,NaN,1400 S DE LA TORRE WY,NaN,34.0166,-118.2058,01,21,2021,12:00:00,AM
276580,212000771,05/28/2021 12:00:00 AM,1930,20,Olympic,2056,2,888,TRESPASSING,0202 1402 1822,...,NaN,900 S BERENDO ST,NaN,34.0527,-118.2937,05,28,2021,12:00:00,AM
276581,211208872,03/19/2021 12:00:00 AM,1105,12,77th Street,1218,1,510,VEHICLE - STOLEN,NaN,...,NaN,58TH ST,FIGUEROA ST,33.9897,-118.2827,03,19,2021,12:00:00,AM
276582,210506531,03/04/2021 12:00:00 AM,2210,5,Harbor,564,2,434,FALSE IMPRISONMENT,0319 0334 0400 0443 1814 2000,...,NaN,200 W 2ND ST,NaN,33.7424,-118.2814,03,04,2021,12:00:00,AM


In [17]:
query="""
SELECT  Year,Month, COUNT(*) AS 'Crime Count'
FROM crime_data
GROUP BY 1,2
"""
crime_count=ps.sqldf(query)
crime_count

,Year,Month,Crime Count
0,2020,01,16058
1,2020,02,16488
2,2020,03,15541
3,2020,04,15176
4,2020,05,16330
5,2020,06,16966
6,2020,07,16912
7,2020,08,16605
8,2020,09,15639
9,2020,10,16143


In [18]:
query="""
SELECT 
  (CASE 
    WHEN AM_PM LIKE 'AM%' THEN 'AM'
    ELSE 'PM'
  END) AS Time, 
  COUNT(*) AS `Crime Count`
FROM crime_data
GROUP BY Time
ORDER BY `Crime Count` DESC;
"""
names_counts=ps.sqldf(query)
names_counts

,Time,Crime Count
0,AM,276584


In [19]:
query="""
SELECT Time,COUNT(*)
FROM crime_data
GROUP BY 1;
"""
tim_count=ps.sqldf(query)
tim_count

,Time,COUNT(*)
0,12:00:00,276584


### Now it seems we have one specific time of day everything is recorded and for that we will drop everything related to the time

In [20]:
crime_data.drop(columns=['DATE OCC','Time','AM_PM'],inplace=True)


### Counting the Percentage of male to female crimes 

In [21]:
query="""
SELECT (CASE
WHEN `Vict Sex` like '%M%' THEN 'M'
WHEN `Vict Sex` like '%F%' THEN 'F'
ELSE 'Un-Known'
 END) AS Gender, COUNT(*) AS 'Crime Exposure'
FROM crime_data
GROUP BY 1;
"""
sex_t_vict=ps.sqldf(query)
sex_t_vict

,Gender,Crime Exposure
0,F,99578
1,M,117711
2,Un-Known,59295


In [22]:
query="""
Select `Crm Cd Desc`, Year, Count(*) as 'Occurrence count'
FROM crime_data
GROUP BY 1,2;
"""
crime_desc=ps.sqldf(query)
crime_desc

,Crm Cd Desc,Year,Occurrence count
0,ARSON,2020,660
1,ARSON,2021,289
2,ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER,2020,397
3,ASSAULT WITH DEADLY WEAPON ON POLICE OFFICER,2021,90
4,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",2020,11494
...,...,...,...
248,VIOLATION OF RESTRAINING ORDER,2021,1093
249,VIOLATION OF TEMPORARY RESTRAINING ORDER,2020,153
250,VIOLATION OF TEMPORARY RESTRAINING ORDER,2021,76
251,WEAPONS POSSESSION/BOMBING,2020,9


In [23]:
query="""
Select `Crm Cd Desc`, Count(*) as 'Occurrence count'
FROM crime_data
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 5;
"""
desc_count=ps.sqldf(query)
desc_count

,Crm Cd Desc,Occurrence count
0,VEHICLE - STOLEN,30046
1,BATTERY - SIMPLE ASSAULT,22528
2,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",18299
3,BURGLARY FROM VEHICLE,17922
4,BURGLARY,17580


In [24]:
fig = px.bar(desc_count, 
             x='Occurrence count', 
             y="Crm Cd Desc", 
             title="Top 5 Crime Categories by Occurrence",
             labels={"Crm Cd Desc": "Crime Category", "Occurrence count": "Number of Occurrences"},
             orientation='h')

# Optionally resize the figure for better spacing (optional)
fig.update_layout(width=900, height=500)

fig.show()


In [25]:
crime_data

,DR_NO,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,Mocodes,Vict Age,...,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON,Month,Day,Year
0,10304468,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,0444 0913,36,...,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978,01,08,2020
1,190101086,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,0416 1822 1414,25,...,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545,01,02,2020
2,201220752,1230,12,77th Street,1259,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),2004 1820 0913 0329 1202,62,...,NaN,NaN,NaN,700 E 73RD ST,NaN,33.9739,-118.2630,09,16,2020
3,191501505,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0329 1402,76,...,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019,01,01,2020
4,191921269,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,31,...,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468,01,01,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276579,210404776,1600,4,Hollenbeck,497,1,510,VEHICLE - STOLEN,NaN,0,...,NaN,NaN,NaN,1400 S DE LA TORRE WY,NaN,34.0166,-118.2058,01,21,2021
276580,212000771,1930,20,Olympic,2056,2,888,TRESPASSING,0202 1402 1822,29,...,998.0,NaN,NaN,900 S BERENDO ST,NaN,34.0527,-118.2937,05,28,2021
276581,211208872,1105,12,77th Street,1218,1,510,VEHICLE - STOLEN,NaN,0,...,NaN,NaN,NaN,58TH ST,FIGUEROA ST,33.9897,-118.2827,03,19,2021
276582,210506531,2210,5,Harbor,564,2,434,FALSE IMPRISONMENT,0319 0334 0400 0443 1814 2000,41,...,NaN,NaN,NaN,200 W 2ND ST,NaN,33.7424,-118.2814,03,04,2021


In [28]:
query="""
Select `AREA NAME`, Count(*) as 'Occurrence count', `Year`
FROM crime_data
GROUP BY 1,3
ORDER BY 2 DESC 
"""
area_count=ps.sqldf(query)
area_count

,AREA NAME,Occurrence count,Year
0,77th Street,12915,2020
1,Central,11281,2020
2,Pacific,11131,2020
3,Southwest,10838,2020
4,Southeast,10543,2020
5,Hollywood,9827,2020
6,Newton,9722,2020
7,N Hollywood,9721,2020
8,Olympic,9337,2020
9,Wilshire,8933,2020


In [41]:
query="""
SELECT `Year`||'/'||`Month` AS 'Time',COUNT(*) AS 'Occurrence Count'
FROM crime_data
GROUP BY 1
ORDER BY 1,2;
"""
desc_count=ps.sqldf(query)
desc_count.drop(desc_count.tail(1).index,inplace=True)
desc_count

,Time,Occurrence Count
0,2020/01,16058
1,2020/02,16488
2,2020/03,15541
3,2020/04,15176
4,2020/05,16330
5,2020/06,16966
6,2020/07,16912
7,2020/08,16605
8,2020/09,15639
9,2020/10,16143


In [42]:
fig=px.line(desc_count,x='Time',y='Occurrence Count',title='Crime Count Over Time')
fig.show()